In [28]:
# !pip install pypdf3
# !pip install tabula-py
# !pip install camelot-py

import pandas as pd
import numpy as np
# import camelot as camel
from PyPDF3 import PdfFileReader 
from tabula import read_pdf 
# from tabulate import tabulate

from my_modules import my_functions as mybib
from my_modules import project_functions as pr

# progress bar 
from tqdm import tqdm

%load_ext autoreload
%autoreload 2


# https://www.youtube.com/watch?v=702lkQbZx50 - for tabula
# https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates - for tabula 

#FIXME: ADDRESS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os 
import re


In [31]:
#FIXME: close pdf file when error occurs
# https://stackoverflow.com/questions/43259717/progress-bar-for-a-for-loop-in-python-script - progress bar
# https://stackoverflow.com/questions/47021185/is-there-a-way-to-close-the-file-pdffilereader-opens - context manager 
# https://www.youtube.com/watch?v=OZo2HxoIOtw - open path as binary 


from tqdm import tqdm

def extract_all_pages_from_pdf(path, file_name, counter, lenght):
    with open(path, 'rb') as f:
        # open path as binary / rb = read binary -> used for pdf files
        counter += 1        
        pdf = PdfFileReader(f)
        last_page = pdf.getNumPages()
        last_page += 1

        # defining page range for import 
        [i for i in range(0, 15)]
        page_range = [i for i in range(5, last_page)]
        # print(page_range)

        # prepare column name filler
        column_names = [i for i in range(0, 14)]
        # print(column_names)

        # create empty dataframe
        adress_data = pd.DataFrame(columns = column_names)
        # display(adress_data)
        
        # extract district information 
        #TODO: nochmal prüfen und vielleicht optimieren
        district = [re.findall(r'(?<=adr_)(.*?)(?=_\d{4}\.pdf)', file_name)]
        print(f'file {counter}/{lenght} - {district[0][0]}')
        for page in tqdm(page_range):
            
            import_data = read_pdf(path, pages = page, encoding = 'ISO-8859-1', stream = True, area = [175, 33, 783, 520], guess = True, pandas_options={'header': None})
            table_df = import_data[0]

            
            table_df = import_data[0]
            columns_len = len(table_df.columns)

            if columns_len < 14:
                if table_df.iloc[:, 3].dtype in ['int64', 'int32', 'float64', 'float32']:
                    table_df.insert(3, 'm1', np.nan)

                if table_df.iloc[:, 6].dtype in ['int64', 'int32', 'float64', 'float32']:
                    table_df.insert(5, 'm2', np.nan)
                
                
                column_length = len(table_df.columns)

                
                if column_length < 14:
                    table_df['m3'] = np.nan
                    # display(table_df)
                
                
                table_df.columns = column_names
                table_df['14'] = str(district[0][0])
                
                adress_data = pd.concat([adress_data, table_df], ignore_index=True , axis=0)

        return adress_data, district[0][0], counter




In [32]:



def get_adress_data_by_year(year):
    path = f'../data/input/berlin_adresses/{year}'
    
    list_of_files = os.listdir(path)
    list_of_files
    
    length = len(list_of_files)
    counter = 0 
    
    column_names = [i for i in range(0, 14)]
    print(len(column_names))
    final_df = pd.DataFrame(columns = column_names)
    
    print(f'Extracting data from files in {path}')
    
    for file_name in (list_of_files):
        path = f'../data/input/berlin_adresses/{year}/{file_name}'
        df, name, counter = extract_all_pages_from_pdf(path, file_name, counter, length)
        
        df.to_csv(f'../data/output/temp_adress_data/{name}-{year}.csv', index = False)
        
        final_df = pd.concat([final_df, df], ignore_index=True , axis=0)
        
        # display(dataset.head(5))
        # print(dataset.shape)
    return final_df




In [33]:
%%time 
total_df = get_adress_data_by_year(2023)

14
Extracting data from files in ../data/input/berlin_adresses/2023
file 1/12 - charlottenburg-wilmersdorf


 18%|██████████████▉                                                                    | 9/50 [00:19<01:27,  2.14s/it]


KeyboardInterrupt: 

In [ ]:
total_df.to_pickle("../data/output/temp_adress_data/address_data.pkl")
total_df.to_csv("../data/output/temp_adress_data/address_data.csv", index=False)


In [ ]:
total_df.columns